# Initialize libraries

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow

from sklearn import decomposition, linear_model,metrics
from sklearn.decomposition import KernelPCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
class_labels = LabelEncoder()
from sklearn.model_selection import cross_val_score,GridSearchCV,StratifiedKFold,KFold,train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import classification_report, confusion_matrix,mean_squared_error,r2_score
from sklearn.metrics import auc, RocCurveDisplay, roc_curve, f1_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.manifold import TSNE
from sklearn.naive_bayes import GaussianNB

## Additional imports from DWI code
import math
from itertools import product
from contextlib import redirect_stdout
import pandas as pd
import time
import scipy
from scipy import io, stats
#from astropy.stats import jackknife_resampling, jackknife_stats, binom_conf_interval
#import xgboost as xgb

seed_value= 42
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
np.random.seed(seed_value)

# Load features

In [4]:
processed_data_path="../../data/processed"

fmri_features=pd.read_csv(f"{processed_data_path}/fMRI/fMRI_features_AAL.csv",index_col=0)

print('fMRI Subject IDs')
print(fmri_features["Subject"])

# dwi_features = pd.read_csv(f"{processed_data_path}/DWI/IDs+Labels+Features.csv")

## Old sub_list (14) above is kept for sample results later,
## new sub_list (22) is loaded below
dwi_features = pd.read_csv(f"{processed_data_path}/DWI/IDs+Labels+Features_AllSubs.csv")
dwi_features["Subject"]=dwi_features["ID"].str[:9]
dwi_features["Late Seizure Label"]=dwi_features["Label"]
dwi_features=dwi_features.drop("Label",axis=1)

print("DWI Subject IDs")
print(dwi_features["Subject"])

eeg_features=pd.read_csv(f"{processed_data_path}/EEG/EEG_features_v0.csv",index_col=0)
print("EEG Subject IDs")
print(eeg_features["Subject"])

fMRI Subject IDs
0     3_13_0063
1     3_13_0068
2     3_16_0013
3     3_16_0016
4     3_16_0023
5     3_16_0033
6     3_16_0036
7     3_17_0001
8     3_17_0004
9     3_17_0007
10    3_17_0009
11    3_17_0012
12    3_17_0019
13    3_17_0048
14    3_19_0050
15    3_21_0040
16    3_21_0061
17    3_26_0080
18    3_26_0092
Name: Subject, dtype: object
DWI Subject IDs
0     3_13_0063
1     3_13_0068
2     3_16_0013
3     3_16_0021
4     3_16_0027
5     3_16_0033
6     3_16_0036
7     3_16_0038
8     3_17_0001
9     3_17_0003
10    3_17_0004
11    3_17_0005
12    3_17_0007
13    3_17_0009
14    3_17_0012
15    3_17_0019
16    3_17_0030
17    3_17_0048
18    3_21_0040
19    3_21_0061
20    3_24_0035
21    3_26_0100
Name: Subject, dtype: object
EEG Subject IDs
0    3_17_0001
1    3_17_0003
2    3_17_0004
3    3_17_0007
4    3_17_0009
5    3_17_0012
6    3_17_0019
7    3_17_0031
8    3_17_0048
9    3_21_0076
Name: Subject, dtype: object


In [5]:
# basic check for correctness
#fMRI, EEG use "Subject" , "Late Seizure Label"

for row_id,row in fmri_features.iterrows():
    fmri_label=row["Late Seizure Label"]
    eeg_label=[]
    if any(eeg_features["Subject"]==row["Subject"]):
        eeg_label=int(eeg_features["Late Seizure Label"].loc[eeg_features["Subject"]==row["Subject"]].to_numpy()[0])
        if fmri_label!=eeg_label:
            print(f'fMRI EEG mismatch subject {row["Subject"]}')

    if any(dwi_features["Subject"]==row["Subject"]):
        dwi_label=int(dwi_features["Late Seizure Label"].loc[dwi_features["Subject"]==row["Subject"]].to_numpy()[0])
        if fmri_label!=dwi_label:
            print(f'fMRI DWI mismatch subject {row["Subject"]}')
    


In [6]:
# need to load EEG and DWI features and sort out which subjects to use programatically
all_features_df=fmri_features.set_index("Subject").join(dwi_features.set_index("Subject"),how="outer",lsuffix=" fMRI",rsuffix=" DWI").reset_index()
all_features_df=all_features_df.set_index("Subject").join(eeg_features.set_index("Subject"),how="outer",lsuffix=" Mix",rsuffix=" EEG").reset_index()
all_features_df["Late Seizure Label EEG"]=all_features_df["Late Seizure Label"]


all_features_df["Late Seizure Label"]=(all_features_df["Late Seizure Label fMRI"].fillna(0)+all_features_df["Late Seizure Label DWI"].fillna(0)+all_features_df["Late Seizure Label EEG"].fillna(0))>0


In [7]:
def remove_non_features(column_list):
    '''Removes column names that aren't features from column list'''
    for to_remove in ["ID","Late Seizure Label","Subject","Subject Number"]:
        if to_remove in column_list:
            column_list.remove(to_remove)
    return column_list

In [10]:

# make np array features for classification

dwi_columns=remove_non_features([*dwi_features])
eeg_columns=remove_non_features([*eeg_features])
fmri_columns=remove_non_features([*fmri_features])

#dwi 
id_subs = dwi_features["ID"].to_numpy()
Y = dwi_features["Late Seizure Label"].to_numpy()
X = dwi_features[dwi_columns].to_numpy()

# fMRI
y=fmri_features["Late Seizure Label"].to_numpy()

overlap_columns=[]
for col in fmri_columns:
    if "Overlap AAL" in col:
        overlap_columns.append(col)

x_over_aal=fmri_features[overlap_columns].to_numpy()


[1 0 1 1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1]


# Baseline fMRI classifier function

In [60]:
def fmri_classifier(x,y,n_subs,cv_inner,cv_outer,score_string,feature_string):
    ''' Prints performance based on nested CV of kPCA combined with SVC for x and y.
    '''    
    seed_value= 42
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
        
    pipe_svc=Pipeline([("scale",StandardScaler()),("pca",KernelPCA()),("svm",SVC(probability=True))])
    param_grid_svc={"pca__n_components":[2,3,4,5,6],"pca__gamma":[.01,.05,.1],"pca__kernel":["sigmoid","rbf"],
    "svm__C": [1, 10, 100], "svm__gamma": [.01, .1]}

    search_svc=GridSearchCV(estimator=pipe_svc,scoring=score_string,param_grid=param_grid_svc,cv=cv_inner,refit=True)
    
#     scores_svc = cross_val_score(search_svc, x, y, scoring=score_string, cv=cv_outer, n_jobs=-1
#     print(f"Mean {scores_svc.mean()} and STD {scores_svc.std()}")

## Below excerpts are added to collect train and test predictions from the fMRI classifier

    fold_no = cv_outer
    data_type  = np.float32
    X_fmri = np.zeros((fold_no, n_subs), dtype = data_type) 
    f1_scores = []
    
    folds = StratifiedKFold(n_splits=fold_no, shuffle=True, 
                            random_state=seed_value).split(x, y)

    for j, (train_idx, test_idx) in enumerate(folds):
        X_train_CV = x[train_idx,:]  
        Y_train_CV = y[train_idx]   
        X_test_CV = x[test_idx,:]   
        Y_test_CV = y[test_idx]  
        
        ## 'model' is cleared here, should there be differnt models desired at each fold
        model = None
        ## The GridSearchCV selected model is passed here
        model = search_svc
        model.fit(X_train_CV, Y_train_CV)         
   
        ## Output probabilities are collected for both labels        
        y_train_prob = model.predict_proba(X_train_CV) 
        y_test_prob = model.predict_proba(X_test_CV)  

        ## Output probabilities for label '1' are collected as 'soft' labels   
        y_train_soft = y_train_prob[:,1]
        y_test_soft = y_test_prob[:,1]

        for n in range(len(y_train_soft)):
            X_fmri[j,n] = y_train_soft[n]
        for q in range(len(y_test_soft)):
            X_fmri[j,n+q+1] = y_test_soft[q]

        ## Output probabilities are thresholded for performance evaluation   
        y_train_pred = [i if i>0.5 else 0 for i in y_train_soft]
        y_test_pred = [1 if i>0.5 else 0 for i in y_test_soft]
                
        f1_scores.append(f1_score(Y_test_CV, y_test_pred, average='weighted'))

    f1_scores = np.array(f1_scores)
    print(feature_string,'\n',score_string,'Score:')
    print(f"Mean {f1_scores.mean()} and STD {f1_scores.std()}")
    
    ## Returns a 'soft' label prediction array, which is [fold_no * no_of_subjects]
    return X_fmri

# Call to the baseline fMRI classifier

In [63]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

n_subs = 14 # No. of subjects
cv_outer=5
cv_inner=KFold(n_splits=3,shuffle=True,random_state=42)

X_fmri = fmri_classifier(x_over_aal,y,n_subs,cv_inner,cv_outer,"f1","Overall Strength AAL")

print('\nSoft labels from fMRI: \n', X_fmri)

Overall Strength AAL 
 f1 Score:
Mean 0.5866666666666667 and STD 0.22070593809662464

Soft labels from fMRI: 
 [[0.61344963 0.5867281  0.5894009  0.6131229  0.6131252  0.6144325
  0.60970795 0.04414539 0.6201913  0.61643404 0.8392137  0.5
  0.612655   0.59729505]
 [0.5851145  0.6383988  0.7107041  0.5497094  0.7062056  0.63839066
  0.73379594 0.6955469  0.9744982  0.57318723 0.89217496 0.7586372
  0.759464   0.6235816 ]
 [0.79753786 0.8751962  0.5867876  0.79753417 0.75504684 0.59968156
  0.62722456 0.08027853 0.08028168 0.8004634  0.7973967  0.61028075
  0.58336693 0.79900354]
 [0.5889551  0.7921351  0.58901405 0.5159537  0.7337308  0.5890122
  0.5635443  0.857438   0.9677468  0.5526291  0.58883667 0.67198086
  0.7296655  0.999996  ]
 [0.59561974 0.56444657 0.6446398  0.4597996  0.6274126  0.5957533
  0.6129569  0.569433   0.68472975 0.60838765 0.5956206  0.5500343
  0.57045925 0.26739478]]


# Baseline DWI classifier function

In [35]:
def dwi_classifier(X,Y,n_subs,n_feats,cv_outer,score_string,feature_string):
    ''' Prints classification performance and collects prediction probabilities for x and y.
    '''    
    seed_value= 42
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)

    fold_no = cv_outer
    data_type  = np.float32
    X_dwi = np.zeros((fold_no, n_subs), dtype = data_type) 

    f1_scores = []
    folds = StratifiedKFold(n_splits=fold_no, shuffle=True, 
                            random_state=seed_value).split(X, Y)

    for j, (train_idx, test_idx) in enumerate(folds):
        X_train_CV = X.iloc[train_idx,:]  
        Y_train_CV = Y.iloc[train_idx]   
        Y_train_CV = np.ravel(Y_train_CV)
        X_test_CV = X.iloc[test_idx,:]   
        Y_test_CV = Y.iloc[test_idx]  
        Y_test_CV = np.ravel(Y_test_CV)
        
        ## Univariate feature selection
#         sel_mutual = SelectKBest(mutual_info_classif, k=n_feats)
        sel_mutual = SelectKBest(chi2, k=n_feats)
#         sel_mutual = SelectKBest(f_classif, k=n_feats)

        X_train_CV = sel_mutual.fit_transform(X_train_CV, Y_train_CV)
        X_test_CV = sel_mutual.transform(X_test_CV)

        model = None
#         model = LinearDiscriminantAnalysis()      
        model = AdaBoostClassifier(n_estimators=100)
        model.fit(X_train_CV, Y_train_CV)
   
        ## Output probabilities are collected for both labels        
        y_train_prob = model.predict_proba(X_train_CV) 
        y_test_prob = model.predict_proba(X_test_CV)  

        ## Output probabilities for label '1' are collected as 'soft' labels   
        y_train_soft = y_train_prob[:,1]
        y_test_soft = y_test_prob[:,1]

        for n in range(len(y_train_soft)):
            X_dwi[j,n] = y_train_soft[n]
        for q in range(len(y_test_soft)):
            X_dwi[j,n+q+1] = y_test_soft[q]

        ## Output probabilities are thresholded for performance evaluation   
        y_train_pred = [i if i>0.5 else 0 for i in y_train_soft]
        y_test_pred = [1 if i>0.5 else 0 for i in y_test_soft]
                
        f1_scores.append(f1_score(Y_test_CV, y_test_pred, average='weighted'))

    f1_scores = np.array(f1_scores)
    print('\n',feature_string,'Classifier,',score_string,'Score:')
    print(f"Mean {f1_scores.mean()} and STD {f1_scores.std()}")
    
    ## Returns an array of 'soft' labels, which is [fold_no * no_of_subjects]
    return X_dwi

# Call to the baseline DWI classifier

In [64]:
n_subs = 14
cv_outer = 5
n_feats = 7

X_dwi = dwi_classifier(X,Y,n_subs,n_feats,cv_outer,"f1","chi2-AdaBoost")

print('\nSoft labels from DWI: \n', X_dwi)


 chi2-AdaBoost Classifier, f1 Score:
Mean 0.9333333333333332 and STD 0.13333333333333336

Soft labels from DWI: 
 [[9.9999350e-01 6.4875835e-06 9.9999619e-01 9.9999607e-01 6.4875835e-06
  6.4875835e-06 6.4875835e-06 9.9999475e-01 9.9999350e-01 9.9999350e-01
  9.9999619e-01 9.9920315e-01 2.6234302e-01 9.9458814e-01]
 [9.9985677e-01 9.9994808e-01 9.9989462e-01 1.4123785e-04 9.9988711e-01
  9.9989873e-01 1.4123785e-04 1.4123785e-04 1.4123785e-04 1.0000000e+00
  9.9985677e-01 2.7744222e-02 1.4997989e-01 9.9999946e-01]
 [9.9999398e-01 5.3810595e-06 9.9999398e-01 6.6243315e-06 9.9999398e-01
  9.9999398e-01 5.3810595e-06 7.8677995e-06 9.9999213e-01 9.9999398e-01
  9.9999362e-01 1.0000000e+00 7.8677995e-06 9.9999124e-01]
 [6.5000377e-06 9.9999350e-01 9.9999756e-01 6.5000377e-06 9.9999350e-01
  9.9999619e-01 6.5000377e-06 6.5000377e-06 9.9999475e-01 9.9999350e-01
  9.9999350e-01 9.9997252e-01 2.9343336e-03 9.2363250e-01]
 [9.9999350e-01 6.5097142e-06 9.9999350e-01 9.9999774e-01 6.5097142e-06
 

# Baseline EEG classifier function

In [ ]:
## def eeg_classifier():
##     return X_eeg

# Call to the baseline EEG classifier

In [ ]:
## X_eeg = eeg_classifier()

# Baseline fusion classifier 

In [67]:
seed_value= 42
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

n_modalities = 2 # No. of modalities for fusion 
cv_outer = 5
n_subs = 14
data_type  = np.float32   
f1_scores = []
X_fusion = np.zeros((cv_outer,n_subs,n_modalities), dtype = data_type) 

## Meta labels are loaded
X_fusion[:,:,0] = X_fmri
X_fusion[:,:,1] = X_dwi
# X_fusion[:,:,2] = X_eeg

## Due to small size of each fold, 
## certain evaluation metrics and plots are currently omitted

folds = StratifiedKFold(n_splits=cv_outer, shuffle=True, 
                        random_state=seed_value).split(X,Y) 

for j, (train_idx, test_idx) in enumerate(folds):
    X_train_CV = X_fusion[j,0:len(train_idx),:]
    Y_train_CV = Y.iloc[train_idx]   
    Y_train_CV = np.ravel(Y_train_CV)
    X_test_CV = X_fusion[j,len(train_idx):,:]   
    Y_test_CV = Y.iloc[test_idx]     
    Y_test_CV = np.ravel(Y_test_CV)

    model_1 = LogisticRegression(random_state=seed_value)   
#     model_2 = GaussianNB()         
#     model_3 = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
#     model_4 = AdaBoostClassifier(n_estimators=100)
#     model_fuse = VotingClassifier(estimators=[('lr',model_1),('gnb',model_2),
#                                               ('svc',model_3),('adb',model_4)],voting='soft')

    model_1.fit(X_train_CV, Y_train_CV)                      
    y_test_pred = model_1.predict(X_test_CV)                  
    f1_scores.append(f1_score(Y_test_CV, y_test_pred, average='weighted'))

f1_scores = np.array(f1_scores)
print('Fusion classifier \n F1 Score:')
print(f"Mean {f1_scores.mean()} and STD {f1_scores.std()}")

Fusion classifier 
 F1 Score:
Mean 0.8400000000000001 and STD 0.20044395171163878


# Dump

In [ ]:
# n_feats = [i+1 for i in range(10)]
# for i in n_feats:
#     X_dwi = dwi_classifier(X,Y,n_subs,i,cv_outer,"f1","chi2-LDA")